In [ ]:
from __future__ import print_function 

#%load_ext autoreload
#%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist,mnist_m
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
#from experiments.SL_bound import *
from util.kl import *
from util.misc import *
from results.plotting import *

# Hyper-parameters
#batch_size = 128
num_classes = 10
make_plots = False

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
    
img_rows, img_cols = 32, 32
project_folder = "/cephyr/users/adambre/Alvis/"

# # check output_dir, create it if not exists
#     if not os.path.isdir(output_dir):
#         os.makedirs(output_dir)

# Load data

In [ ]:
import time
import os
import numpy as np
import pandas as pd



In [ ]:
TASK=4
x_source,y_source,x_target,y_target=load_task(TASK)

## print amount of each labels in the dataset
print(len(y_source))
print(np.sum(y_source,axis=0))
print("-"*40)
print(len(y_target))
print(np.sum(y_target,axis=0))

#data_path="/home/adam/Code/Datasets/chexpert/1.0.0/"
#data_path2="/home/adam/Code/Datasets/chexpert/CheXpert-v1.0-small"



In [ ]:
from experiments.training import *



def find_optimal_sigma(sigmas,epsilon, alpha,Binary=False,Task=2):
    #### to find the optimal sigma just do a search through all the results 
    #### and save the one for each parameter which has the minimal bound
    #### Do we do this per epoch or for some other value? The sigma which yields the lowest bound overall for some epoch?
    optimal=[0,1]
    # search through all epochs and pick the sigma which yields the smallest bound during the whole training process
    for sigma in sigmas:
        sigma_tmp=sigma
        sigma=sigma[0]*10**(-1*sigma[1])
        if Binary:
            result_path="results/"+"task"+str(TASK)+"/Binary/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        else:
            result_path="results/"+"task"+str(TASK)+"/"+str(int(1000*epsilon))+"_"+str(int(100*alpha))+"_"+str(sigma_tmp[0])+str(sigma_tmp[1])
        results=pd.read_pickle(result_path+"_results.pkl")
        MIN=np.min(results["Bound"])
        if (MIN<optimal[1]):
            optimal[1]=MIN
            optimal[0]=sigma
    print("The optimal sigma is {} with bound value {}".format(optimal[0],optimal[1]))


# Training

In [ ]:
#alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
alphas=[0.1]#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]#,0.3,0.5,0.7,0.9]
epsilons=[0.01]#,0.01,0.001]
architectures=["lenet"]
tasks=[4]
for t in tasks:
    x_source,y_source,x_target,y_target=load_task(t)
    print(len(y_source))
    print(len(y_target))
    for arch in architectures:
        for alpha in alphas:
            if alpha==0:
                pass
            else:
                print("Alpha is:"+str(alpha))
                x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=69105)
                w_a=train_prior(alpha,1,x_prior,y_prior,x_target=x_target,y_target=y_target,save=True,Task=t,Binary=True,batch_size=128,architecture=arch)
            for epsilon in epsilons:
                w_s=train_posterior(alpha,x_source,y_source,None,x_test=x_source,y_test=y_source,save=True,epsilon=epsilon,Task=t,Binary=True,batch_size=128,architecture=arch)

In [ ]:
#alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
alphas=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]#,0.3,0.5,0.7,0.9]
epsilons=[0.05]#,0.1,0.05]#[0.03,0.01,0.001] [0.15,0.1,0.05]
architectures=["lenet"]
tasks=[6]
for t in tasks:
    """
    task 6
    239886
    [174064  65822]
    ----------------------------------------
    95882
    [94126  1756]
    """
    x_source,y_source,x_target,y_target=load_task(t)
    print(len(y_source))
    print(len(y_target))
    for arch in architectures:
        for alpha in alphas:
            if alpha==0:
                pass
            else:
                print("Alpha is:"+str(alpha))
                x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=69105)
                w_a=train_prior(alpha,1,x_prior,y_prior,x_target=x_target,y_target=y_target,save=True,Task=t,Binary=True,batch_size=128,architecture=arch)
            for epsilon in epsilons:
                w_s=train_posterior(alpha,x_source,y_source,None,x_test=x_source,y_test=y_source,save=True,epsilon=epsilon,Task=t,Binary=True,batch_size=128,architecture=arch)